In [1]:
# Instalar paquetes
!pip install hubspot-api-client
!pip install rut_chile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 34.0 MB/s eta 0:00:00


In [2]:
import requests
from rut_chile import rut_chile as rc
import pandas as pd
import numpy as np
import unicodedata
import re
import time
import psutil
import os

start = time.time()
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss
# Guardar tiempo de inicio
inicio = time.time()



# Función para validar rut
def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception as e:
        return False

def agregar_guion(valor):
    if pd.notna(valor) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    else:
        return valor

# Función para eliminar 0 al principio de una cadena
def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

def consulta_api(id, access_token=None, identificador='rut'):
    # Definir URL y variables según identificador
    if identificador == 'rut':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byRut/{id}/true'
    elif identificador == 'icm':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byICM/{id}/true'
    elif identificador == 'direccion':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/Direcciones/byICM/{id}'
    elif identificador == 'datos_direccion':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/Direcc/byICM/{id}'

    # Obtener token si no se proporciona
    if access_token is None:
        access_token = autorizacion_api()

    # Configurar encabezados
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        # Realizar la solicitud
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Lanza una excepción para códigos HTTP >= 400

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            return {
                "error": "Recurso no encontrado.",
                "status": 404,
                "detalle": f"No se encontró el recurso solicitado con ID: {id}. URL: {url}"
            }
        elif response.status_code == 401:
            return {
                "error": "No autorizado.",
                "status": 401,
                "detalle": "El token es inválido o ha expirado. Intenta obtener un nuevo token."
            }
        else:
            return {
                "error": f"HTTP Error {response.status_code}",
                "detalle": response.text
            }
    except requests.exceptions.RequestException as req_err:
        return {
            "error": "Error de conexión.",
            "detalle": str(req_err)
        }

    # Si no hay errores, retorna el contenido como JSON
    return response.json()

def autorizacion_api():
    # URL de autenticación
    url_auth = 'https://authentication.colegiomedico.cl/api/authentication/authenticate'

    # Credenciales desde variables de entorno
    user = 'UsersCRSantiago'  # Valor por defecto
    psw = 'UsersCRSantiago2024_&_&'  # Valor por defecto

    # Verificar credenciales
    if not user or not psw:
        raise ValueError("Las credenciales de usuario o contraseña no están configuradas.")

    # Datos de autenticación
    data = {"userName": user, "password": psw}

    # Encabezados
    headers = {"Content-Type": "application/json"}

    # Solicitar token
    response_auth = requests.post(url_auth, headers=headers, json=data)

    # Verificar respuesta
    if response_auth.status_code != 200:
        raise Exception(f"Error al autenticar: {response_auth.status_code} - {response_auth.text}")

    # Retornar token
    access_token = response_auth.text
    return access_token.strip('"')

#recorrer la api del nacional desde un id_inical hasta un id_final
def obtener_datos_desde_api(id_inicial, id_final, identificador='icm'):
    resultados = []

    for id_actual in range(id_inicial, id_final + 1):
        try:
            print(f"Consultando ID: {id_actual}")
            # Llama a la función consulta_api (definida previamente)
            respuesta = consulta_api(str(id_actual), identificador=identificador)
            resultados.append(respuesta)

        except Exception as e:
            print(f"Error con ID {id_actual}: {e}")
            continue

    # Convertir resultados en un DataFrame
    return pd.DataFrame(resultados)


def limpiar_valor(valor):
    """Elimina caracteres especiales, convierte a minúsculas, y elimina tildes"""
    # Eliminar tildes y acentos
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()

# Función para validar email
def es_correo_valido(email):
    # Verificar si el email es NaN o vacío
    if pd.isna(email) or email.strip() == '':
        return False

    # Patrón de la expresión regular para un email válido
    patron = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    # Usar re.match para comprobar si el email coincide con el patrón
    if re.match(patron, email):
        return True
    else:
        return False

def obtener_direcciones(df_o_lista):
    resultados = []
    contador = 0  # Contador de consultas

    # Verificar si la entrada es un DataFrame o una lista
    if isinstance(df_o_lista, pd.DataFrame):
        if 'idMedico' not in df_o_lista.columns:
            print("Error: El DataFrame no contiene la columna 'idMedico'.")
            return pd.DataFrame()
        ids = df_o_lista['idMedico'].dropna().unique()
    elif isinstance(df_o_lista, list):
        ids = list(set(df_o_lista))  # Eliminar duplicados
    else:
        print("Error: La entrada debe ser un DataFrame o una lista de IDs.")
        return pd.DataFrame()

    total_ids = len(ids)
    print(f"Total de IDs a procesar: {total_ids}")

    for id_medico in ids:
        contador += 1  # Incrementa el contador por cada consulta

        try:
            datos = consulta_api(id_medico, identificador='direccion')
        except Exception as e:
            print(f"Error en la consulta de ID {id_medico}: {e}")
            continue

        # Depuración: Mostrar progreso y datos recibidos
        print(f"Consulta {contador}/{total_ids} - ID: {id_medico}, Tipo de datos retornado: {type(datos)}")

        if isinstance(datos, list):
            resultados.extend(datos)
        else:
            print(f"Advertencia: La API devolvió un {type(datos)} en lugar de una lista para ID {id_medico}")

    # Convertir los resultados a un DataFrame si hay datos válidos
    df_resultado_direcciones = pd.DataFrame(resultados) if resultados else pd.DataFrame()

    print(f"Consultas finalizadas. Total procesados: {contador}, Filas obtenidas: {df_resultado_direcciones.shape[0]}")

    return df_resultado_direcciones

# Función para determinar el valor de 'email_final' basado en las reglas dadas
def determinar_email_final(row):
    if row['email_nulo_o_vacio']:
        return 'Nulo'
    elif not row['email_valido']:
        return 'Invalido'
    elif row['email_duplicado']:
        return 'Duplicado'
    else:
        return 'Valido'

# Función para limpiar y formatear el número de celular en Chile
def limpiar_y_formatear_valor(celular):
    if pd.isna(celular) or celular is None:
        return ''

    # Limpiar el valor dejando solo números
    celular = re.sub(r'\D', '', str(celular))

    if len(celular) >= 8:
        return f"569{celular[-8:-4]}{celular[-4:]}"
    return celular

# Función para validar un número de celular chileno
def validar_celular_chile(numero):
    if not isinstance(numero, str) or pd.isna(numero):
        return False

    # Permitir números que comiencen con "+569" o "569" y que tengan 9 dígitos en total después del 9
    patron = re.compile(r"^(?:\+?56)?9\d{8}$")
    return bool(patron.fullmatch(numero))

# Función para determinar el valor de 'rcm_final' basado en las reglas dadas
def determinar_celular_final(row):
    if row['celular_nulo_o_vacio']:
        return 'Nulo'
    elif row['celular_duplicado']:
        return 'Duplicado'
    elif row['mobilephone_valido']:
        return 'Valido'
    else:
        return 'Invalido'

In [ ]:
from hubspot import HubSpot
api_client = HubSpot(access_token='your_access_token')

# or set your access token later
api_client = HubSpot()
api_client.access_token = os.getenv('HUBSPOT_API_KEY')

#Especifica las propiedades que deseas recuperar
properties = [ 'address','firstname','segundo_nombre', 'lastname','apellido_materno', 'condicion', 'createdate','crs',
            'date_of_birth','direccion_de_trabajo','edad','email','estado_pago','fecha_inscripcion_al_colegio',
'fecha_titulo','gender','graduation_date','mobilephone','rcm','rut', 'lastmodifieddate', 'lifecyclestage',
              'phone','hs_object_source_label','especialidades','fecha_act_super','fecha_actualizacion_pagos','hs_whatsapp_phone_number']

try:
    # Recupera todos los contactos con las propiedades especificadas
    all_contacts = api_client.crm.contacts.get_all(properties=properties)

    # Convierte cada contacto a un diccionario y almacenarlos en una lista
    contacts_list = [contact.to_dict() for contact in all_contacts]

    # Crear un DataFrame de pandas a partir de la lista de diccionarios
    df_contacts = pd.DataFrame(contacts_list)

except ApiException as e:
    print("Exception when calling contacts API: %s\n" % e)

# Normalizar la columna que contiene diccionarios
df_col_dict = pd.json_normalize(df_contacts['properties'])

# Opcional: unir el DataFrame normalizado con el DataFrame original, excluyendo la columna original
df_final_contactos = df_contacts.drop(columns=['properties']).join(df_col_dict)

# Ordenar por 'Fecha de creación' (descendente) y 'Rut_Normalizado' (ascendente)
df_final_contactos = df_final_contactos.sort_values(by=['createdate'], ascending = True)

In [4]:
#normalizar columna rut
df_final_contactos['rut_normalizado'] = (
    df_final_contactos['rut'].str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero).astype(str))

# Aplicar la función is_valid_rut y crear una nueva columna con los resultados
df_final_contactos['rut_valido'] = df_final_contactos['rut_normalizado'].apply(validar_rut)

# Extraer el RUT sin el dígito verificador y agregarlo a una nueva columna usando
df_final_contactos['rut_sdv'] = df_final_contactos['rut_normalizado'].str.split('-').str[0]

# Filtrar las filas donde 'rut' es True
df_rut_filtrado_crm = df_final_contactos[df_final_contactos['rut_valido'] == True]

columnas_hubspot =['id','rut_normalizado','rut','rcm','rut_sdv','fecha_actualizacion_pagos','email','mobilephone','hs_whatsapp_phone_number','phone']
df_hubspot = df_rut_filtrado_crm[columnas_hubspot]

In [5]:
#Trabajar variable celular

# Agregar una nueva columna que marque si 'email' es nula o vacía
df_hubspot['celular_nulo_o_vacio'] = df_hubspot['hs_whatsapp_phone_number'].isna() | (df_hubspot['hs_whatsapp_phone_number'] == '')
# Aplicar la función de formateo a la columna 'celular_principal'
df_hubspot['mobilephone_formateado'] = df_hubspot['hs_whatsapp_phone_number'].apply(limpiar_y_formatear_valor)
# crear columna con analisis de duplicados segun email para df_contactos
df_hubspot['celular_duplicado'] = df_hubspot.duplicated(subset=['hs_whatsapp_phone_number'], keep='first')

# Aplicar la función al DataFrame y crear la columna 'mobilephone_valido'
df_hubspot['mobilephone_valido'] = df_hubspot['mobilephone_formateado'].apply(validar_celular_chile)

# Aplicar la función a cada fila del DataFrame
df_hubspot['celular_final'] = df_hubspot.apply(determinar_celular_final, axis=1)






/tmp/ipython-input-5-763588599.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hubspot['celular_nulo_o_vacio'] = df_hubspot['hs_whatsapp_phone_number'].isna() | (df_hubspot['hs_whatsapp_phone_number'] == '')
/tmp/ipython-input-5-763588599.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hubspot['mobilephone_formateado'] = df_hubspot['hs_whatsapp_phone_number'].apply(limpiar_y_formatear_valor)
/tmp/ipython-input-5-763588599.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [6]:
df_hubspot

,id,rut_normalizado,rut,rcm,rut_sdv,fecha_actualizacion_pagos,email,mobilephone,hs_whatsapp_phone_number,phone,celular_nulo_o_vacio,mobilephone_formateado,celular_duplicado,mobilephone_valido,celular_final
2,351,17697342-0,17697342-0,37074,17697342,2025-04-23,ja.zavalav@gmail.com,56942655572,+56942655572,None,False,56942655572,False,True,Valido
3,551,12642260-1,12642260-1,22267,12642260,2025-05-13,agatamar@gmail.com,56997441374,+56997441374,None,False,56997441374,False,True,Valido
5,701,5452530-3,5452530-3,10301,5452530,2025-04-22,altamirano.c@gmail.com,56997969399,+56997969399,None,False,56997969399,False,True,Valido
6,751,17327657-5,17327657-5,40205,17327657,2025-04-25,aguileraberrios.dra@gmail.com,56991819326,+56991819326,None,False,56991819326,False,True,Valido
8,851,25660725-5,25660725-5,51241,25660725,2025-04-25,marialbarran915@gmail.com,+56-9-4932-0569,+56-9-4932-0569,+56-9-4932-0569,False,56949320569,False,True,Valido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71655,133661976447,10204181-k,10204181-k,None,10204181,None,edu.quantumr5.0@gmail.com,None,None,None,True,,True,False,Nulo
71657,133664496207,18531614-9,18.531.614-9,None,18531614,None,r.suarezvera@gmail.com,None,None,None,True,,True,False,Nulo
71654,133656089618,13864100-7,13864100-7,0,13864100,None,drapradenas@gmail.com,56934241960,None,None,True,,True,False,Nulo
71658,133668151297,6871992-5,6871992-5,14055,6871992,None,rebecamislevalla555@gmail.com,56999980676,None,None,True,,True,False,Nulo


In [7]:
df_agrupado = df_hubspot.groupby('celular_final').size().reset_index(name='cantidad')
df_agrupado

,celular_final,cantidad
0,Duplicado,609
1,Invalido,283
2,Nulo,16149
3,Valido,50048


In [8]:
df_filtrado_actualizar_celular = df_hubspot[df_hubspot['celular_final'] == 'Nulo']
len(df_filtrado_actualizar_celular)

16149

In [9]:
import concurrent.futures
import pandas as pd
import sys

def obtener_direcciones(df_o_lista):
    resultados = []

    # Verificar si la entrada es un DataFrame o una lista
    if isinstance(df_o_lista, pd.DataFrame):
        if 'idMedico' not in df_o_lista.columns:
            print("Error: El DataFrame no contiene la columna 'idMedico'.")
            return pd.DataFrame()
        ids = df_o_lista['idMedico'].dropna().unique()
    elif isinstance(df_o_lista, list):
        ids = list(set(df_o_lista))  # Eliminar duplicados
    else:
        print("Error: La entrada debe ser un DataFrame o una lista de IDs.")
        return pd.DataFrame()

    total_ids = len(ids)
    print(f"Total de IDs a procesar: {total_ids}")

    def consultar_id(id_medico):
        try:
            datos = consulta_api(id_medico, identificador='direccion')
            return datos if isinstance(datos, list) else None
        except Exception as e:
            print(f"Error en la consulta de ID {id_medico}: {e}")
            sys.stdout.flush()
            return None

    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        for i, resultado in enumerate(executor.map(consultar_id, ids)):
            if resultado:
                resultados.extend(resultado)

            if i % 30 == 0 or i == total_ids - 1:  # Mostrar progreso cada 10 registros
                print(f"Progreso: {i+1}/{total_ids} IDs procesados")
                sys.stdout.flush()

    df_resultado_direcciones = pd.DataFrame(resultados) if resultados else pd.DataFrame()
    print(f"Consultas finalizadas. Total procesados: {total_ids}, Filas obtenidas: {df_resultado_direcciones.shape[0]}")

    return df_resultado_direcciones


In [10]:
#Aplicar funcion Obtener direcciones al df, debe contener una columnas con idMedico
df_filtrado_actualizar_celular['idMedico'] = df_filtrado_actualizar_celular['rcm']
# Ejecutar la función con los primeros 100 registros del DataFrame
y = df_filtrado_actualizar_celular
df_medicos_celular = obtener_direcciones(y)

# Guardar tiempo de inicio
OBTENER_DIRECCIONES = time.time()

/tmp/ipython-input-10-1652685385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_actualizar_celular['idMedico'] = df_filtrado_actualizar_celular['rcm']


Total de IDs a procesar: 7009
Progreso: 1/7009 IDs procesados
Progreso: 31/7009 IDs procesados
Progreso: 61/7009 IDs procesados
Progreso: 91/7009 IDs procesados
Progreso: 121/7009 IDs procesados
Progreso: 151/7009 IDs procesados
Progreso: 181/7009 IDs procesados
Progreso: 211/7009 IDs procesados
Progreso: 241/7009 IDs procesados
Progreso: 271/7009 IDs procesados
Progreso: 301/7009 IDs procesados
Progreso: 331/7009 IDs procesados
Progreso: 361/7009 IDs procesados
Progreso: 391/7009 IDs procesados
Progreso: 421/7009 IDs procesados
Progreso: 451/7009 IDs procesados
Progreso: 481/7009 IDs procesados
Progreso: 511/7009 IDs procesados
Progreso: 541/7009 IDs procesados
Progreso: 571/7009 IDs procesados
Progreso: 601/7009 IDs procesados
Progreso: 631/7009 IDs procesados
Progreso: 661/7009 IDs procesados
Progreso: 691/7009 IDs procesados
Progreso: 721/7009 IDs procesados
Progreso: 751/7009 IDs procesados
Progreso: 781/7009 IDs procesados
Progreso: 811/7009 IDs procesados
Progreso: 841/7009 IDs 

In [11]:
df_medicos_celular

#Trabajar variable celular

# Agregar una nueva columna que marque si 'email' es nula o vacía
df_medicos_celular['celular_nulo_o_vacio'] = df_medicos_celular['fono'].isna() | (df_medicos_celular['fono'] == '')
# Aplicar la función de formateo a la columna 'celular_principal'
df_medicos_celular['fono_formateado'] = df_medicos_celular['fono'].apply(limpiar_y_formatear_valor)
# crear columna con analisis de duplicados segun email para df_contactos
df_medicos_celular['celular_duplicado'] = df_medicos_celular.duplicated(subset=['fono'], keep='first')
# Aplicar la función al DataFrame y crear la columna 'mobilephone_valido'
df_medicos_celular['mobilephone_valido'] = df_medicos_celular['fono_formateado'].apply(validar_celular_chile)

# Aplicar la función a cada fila del DataFrame
df_medicos_celular['celular_final'] = df_medicos_celular.apply(determinar_celular_final, axis=1)


In [12]:
df_actualizar_celular = df_medicos_celular[df_medicos_celular['celular_final'] == 'Valido']
len(df_actualizar_celular)

46

In [13]:

# Convertir 'idMedico' a string (para eliminar espacios o caracteres ocultos)
df_actualizar_celular['idMedico'] = df_actualizar_celular['idMedico'].astype(str).str.strip()
df_hubspot['idMedico'] = df_hubspot['rcm'].astype(str).str.strip()

# Reemplazar valores vacíos con NaN
df_actualizar_celular['idMedico'].replace('', pd.NA, inplace=True)
df_hubspot['idMedico'].replace('', pd.NA, inplace=True)

# Convertir a numérico, forzando NaN en valores no convertibles
df_actualizar_celular['idMedico'] = pd.to_numeric(df_actualizar_celular['idMedico'], errors='coerce')
df_hubspot['idMedico'] = pd.to_numeric(df_hubspot['idMedico'], errors='coerce')

# Eliminar filas con NaN en 'idMedico' antes de convertir a enteros
df_actualizar_celular.dropna(subset=['idMedico'], inplace=True)
df_hubspot.dropna(subset=['idMedico'], inplace=True)

# Convertir a enteros después de eliminar NaN
df_actualizar_celular['idMedico'] = df_actualizar_celular['idMedico'].astype(int)
df_hubspot['idMedico'] = df_hubspot['idMedico'].astype(int)

# Realizar el merge asegurando que 'idMedico' es entero
df_cruce_carga_celular = pd.merge(df_actualizar_celular, df_hubspot, on='idMedico', how='inner')
df_cruce_carga_celular.info()
celular_col =['idMedico','id','rut_normalizado','fono_formateado']
df_cruce_carga_celular_final = df_cruce_carga_celular[celular_col]
df_cruce_carga_celular_final

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   idMedico                   47 non-null     int64  
 1   tipo                       47 non-null     object 
 2   calleyNumero               1 non-null      object 
 3   edificio                   1 non-null      object 
 4   dpto                       34 non-null     object 
 5   comuna                     47 non-null     object 
 6   ciudad                     1 non-null      object 
 7   region                     42 non-null     object 
 8   pais                       1 non-null      object 
 9   fono                       47 non-null     object 
 10  celular                    47 non-null     object 
 11  casilla                    0 non-null      object 
 12  mail                       47 non-null     object 
 13  calle                      47 non-null     object 
 

/tmp/ipython-input-13-3268177314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actualizar_celular['idMedico'] = df_actualizar_celular['idMedico'].astype(str).str.strip()
/tmp/ipython-input-13-3268177314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hubspot['idMedico'] = df_hubspot['rcm'].astype(str).str.strip()
/tmp/ipython-input-13-3268177314.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior

,idMedico,id,rut_normalizado,fono_formateado
0,11188,15673713,7276570-2,56927932640
1,56215,133638420822,19244351-2,56991703550
2,59384,81101034804,19891780-K,56979688376
3,59410,90282873705,20283888-K,56990225823
4,58423,104487295813,20119552-7,56999999999
5,59419,108914913557,19699080-1,56951935568
6,59453,126082467901,19373658-0,56987404329
7,59388,133607470674,28087182-6,56977642293
8,59357,128585219650,26755637-7,56942556411
9,59406,133600783374,18126124-2,56982445754


In [14]:
# Función para formatear el número de celular al formato +56-9-XXXX-XXXX
def formatear_celular(numero):
    numero = re.sub(r'\D', '', str(numero))  # Eliminar caracteres no numéricos
    if len(numero) == 11 and numero.startswith("569"):
        return f"+56-9-{numero[3:7]}-{numero[7:]}"
    return numero  # Si no cumple con el formato esperado, devolver tal cual


# Aplicar la función a la columna 'fono_formateado'
df_cruce_carga_celular_final["fono_final"] = df_cruce_carga_celular_final["fono_formateado"].apply(formatear_celular)


/tmp/ipython-input-14-3682363211.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cruce_carga_celular_final["fono_final"] = df_cruce_carga_celular_final["fono_formateado"].apply(formatear_celular)


In [15]:
len(df_cruce_carga_celular_final)

47

In [ ]:
# Actualizar email segun id Hubspot

import requests

# Configura tu API Key de HubSpot

api_key = os.getenv('HUBSPOT_API_KEY') 

# Supongamos que tienes un DataFrame 'df_contactos' con las columnas: 'id', 'firstname', 'segundo_nombre', 'lastname', 'apellido_materno'
df_contactos = df_cruce_carga_celular_final

# Define los headers
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

# Itera sobre cada fila del DataFrame para actualizar el contacto en HubSpot
for index, row in df_contactos.iterrows():
    contact_id = row['id']
    data = {
        "properties": {
            "hs_whatsapp_phone_number": row['fono_final']
            }
    }

    # Define la URL de la API de HubSpot para actualizar un contacto
    url = f'https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}'

    # Realiza la solicitud PATCH para actualizar el contacto
    response = requests.patch(url, headers=headers, json=data)

    # Verifica la respuesta
    if response.status_code == 200:
        print(f'Contacto con ID {contact_id} actualizado exitosamente')
    else:
        print(f'Error al actualizar el contacto con ID {contact_id}: {response.status_code}')
        print(response.text)

Contacto con ID 15673713 actualizado exitosamente
Contacto con ID 133638420822 actualizado exitosamente
Contacto con ID 81101034804 actualizado exitosamente
Contacto con ID 90282873705 actualizado exitosamente
Contacto con ID 104487295813 actualizado exitosamente
Contacto con ID 108914913557 actualizado exitosamente
Contacto con ID 126082467901 actualizado exitosamente
Contacto con ID 133607470674 actualizado exitosamente
Contacto con ID 128585219650 actualizado exitosamente
Contacto con ID 133600783374 actualizado exitosamente
Contacto con ID 108919593527 actualizado exitosamente
Contacto con ID 108919667801 actualizado exitosamente
Contacto con ID 108919086095 actualizado exitosamente
Contacto con ID 108915345226 actualizado exitosamente
Contacto con ID 108915619719 actualizado exitosamente
Contacto con ID 108915321712 actualizado exitosamente
Contacto con ID 133612083603 actualizado exitosamente
Contacto con ID 108914517841 actualizado exitosamente
Contacto con ID 108915619745 actua

In [17]:
mem_end = process.memory_info().rss
end = time.time()

print(f"Tiempo total: {end - start:.2f} segundos")
print(f"Memoria total usada: {(mem_end - mem_start)/1024**2:.2f} MB")

Tiempo total: 945.10 segundos
Memoria total usada: 464.37 MB
